## Gemini 2.5 Pro: Simple Video Analysis Demo (Files for Video & Reference)

This notebook demonstrates how to use the Gemini API (`gemini-2.5-pro-exp-03-25`) to analyze a video with a reference text file. It’s designed to be simple and clean, focusing on uploading files, processing them, generating an analysis, and showing the results.

In [ ]:
# @title 1. Setup: Import Libraries & Configure API Key
import time
from google.colab import userdata
from google import genai
from IPython.display import Markdown

# Configure API Key (replace 'GOOGLE_API_KEY' with your key name in Colab secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define the model
MODEL_NAME = 'gemini-2.5-pro-exp-03-25'
print(f"Using model: {MODEL_NAME}")

Using model: gemini-2.5-pro-exp-03-25


In [ ]:
# @title 2. Upload Files: Video and Reference Text
def upload_file(client, file_name):
    print(f"Uploading file: {file_name}...")
    try:
        file_obj = client.files.upload(file=file_name)
        print(f"Completed upload: {file_obj.uri}")
        return file_obj
    except FileNotFoundError:
        print(f"Error: File '{file_name}' not found. Please upload it to Colab.")
        return None
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

# File names (change these to your actual files)
video_file_name = 'video.mp4'
reference_file_name = 'reference.txt'

# Upload both files
video_file = upload_file(client, video_file_name)
ref_file = upload_file(client, reference_file_name)

# Ensure both files are uploaded
if not video_file or not ref_file:
    raise ValueError("One or both files failed to upload. Please check and try again.")

Uploading file: video.mp4...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/99zfw6z8tkhr
Uploading file: reference.txt...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/1j6vm4i8dgpn


In [ ]:
# @title 3. Wait for Files to Process
def wait_for_file(client, file_obj):
    if not file_obj:
        return
    print(f"Processing {file_obj.name}", end='')
    while file_obj.state.name == "PROCESSING":
        print('.', end='', flush=True)
        time.sleep(5)  # Check every 5 seconds
        file_obj = client.files.get(name=file_obj.name)
    print(f" Done. State: {file_obj.state.name}")
    if file_obj.state.name == "FAILED":
        raise ValueError(f"File processing failed for {file_obj.name}")

# Wait for both files
wait_for_file(client, video_file)
wait_for_file(client, ref_file)

Processing files/99zfw6z8tkhr.. Done. State: ACTIVE
Processing files/1j6vm4i8dgpn Done. State: ACTIVE


In [ ]:
# @title 4. Define the Analysis Prompt
prompt = """
**Act As:** Expert Video Content Analyst & Editor Assistant

**Goal:** Analyze the draft video to improve cutting efficiency by:
1. Understanding the video’s content and flow.
2. Identifying key segments with clear delivery or important topics, using the reference text file as a context guide.
3. Providing cleaned transcripts (no "ums" or "ahs").

**Input Sources:**
- **Video File:** The draft video (uploaded via API).
- **Reference Text File:** Supplementary context (uploaded via API). Prioritize video content over this.

**Instructions:**
- Watch the entire video to understand its flow and key points.
- Pick segments based on video quality and relevance, using the reference text only to validate topics.
- Clean transcripts by removing filler words without altering meaning.

**Output Format (Markdown):**
```markdown
## Video Analysis for Efficient Cutting

**Cut Segment 1**
* **Timestamp:** [MM:SS - MM:SS]
* **Summary:** [Why this segment stands out.]
* **Transcript:** [Cleaned text.]

**Cut Segment 2**
* **Timestamp:** [MM:SS - MM:SS]
* **Summary:** [Why this segment stands out.]
* **Transcript:** [Cleaned text.]
```
"""

In [ ]:
# @title 5. Generate Analysis
print("Generating analysis (this may take a moment)...")
try:
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=[ref_file, video_file, prompt])
    print("Analysis complete.")
except Exception as e:
    print(f"Error generating analysis: {e}")
    response = None

Generating analysis (this may take a moment)...
Analysis complete.


In [ ]:
# @title 6. Display Results
if response:
    display(Markdown(response.text))
else:
    print("No analysis results generated.")

```markdown
## Video Analysis for Efficient Cutting

**Cut Segment 1**
* **Timestamp:** [0:06 - 1:32]
* **Summary:** Introduces the Gemini 2.5 Pro Experimental model, highlighting its status as Google's best model according to the Chatbot Arena Leaderboard, surpassing competitors like GPT-4o and Grok. Mentions the video will cover using it in AI Studio and via API in a notebook.
* **Transcript:** Hey guys. Hello. Welcome to my channel. And today's topic is about the Gemini 2.5 model, which is released by Google recently, a few days ago. And it turns out to be the most or the best model so far, according to the large language models. And I think today I'm going to walk you through how to use that in the Google AI Studio and also how to call the APIs in a, you know, Python notebook. And if we go to this page, the arena, the arena, like the model performance rankings website, you can see that the Gemini 2.5 Pro is ranked the first, and it's doing the best among the competitors like GPT, Grok, and, you know, Deepseek, other, you know, fancy models. Okay, let's try out the examples.

**Cut Segment 2**
* **Timestamp:** [1:36 - 2:56]
* **Summary:** Shows the Google AI Studio interface for Gemini 2.5 Pro Experimental (03-25 version). Points out the model details including the token count (~1M), confirms it's currently free, highlights the knowledge cutoff (Jan 2025), and crucially contrasts the AI Studio rate limit (5 RPM) with the free tier API rate limit (2 RPM / 50 per day).
* **Transcript:** Okay, here's the Google AI Studio. And if we look at the right, you can see the model is the 2.5. And we can check the input and output pricing. Currently, it's free to use. And you can use for whatever cases, the coding, reasoning, whatever cases. And you can check out the knowledge cut-off is January 2025. And the important thing is that the rate limits. It has five, five requests per minute. For, I think it's for this Google AI Studio inside of that. And for the second one, the two reads per minute, 50 per day, I think that's maybe for the API requests. Right.

**Cut Segment 3**
* **Timestamp:** [2:58 - 3:45]
* **Summary:** Demonstrates selecting the "YouTube video" input option in Google AI Studio, pasting a URL of a previous video about Gemini fine-tuning, and prompting the model for a full analysis.
* **Transcript:** So we can try out some fancy functions here. So, I'm really interested in the YouTube videos. So if I just copy paste my channel, my previous video about the practical Gemini Fine-tuning thing... Alright, it can recognize the video and... So let's try something like, Give me a full analysis of this video.

**Cut Segment 4**
* **Timestamp:** [4:52 - 5:54]
* **Summary:** Shows the successful analysis of the YouTube video generated by Gemini 2.5 Pro within AI Studio, highlighting its ability to understand the video's content, key concepts (SFT, Vertex AI, JSONL, GCS, metrics), process, target audience, and overall impression. Gives positive feedback to the model.
* **Transcript:** Okay, here's a full analysis of the video "Supervised Fine-tuning for Gemini Models". Main Topic & Goal. The video is a practical tutorial demonstrating how to perform supervised fine-tuning on Google's Gemini language models using the Vertex AI platform within Google Cloud... Key Concepts Explained: Supervised Fine-Tuning (SFT) Briefly explained... Gemini Models: The family of Google models being fine-tuned... Vertex AI: Google Cloud's unified ML platform... JSON Lines (JSONL) Format... Google Cloud Storage (GCS)... Training Metrics: Key indicators like "Total Loss"... Process Demonstrated: Introduction & Context... Understanding SFT... Data Preparation... Data Upload... Fine-Tuning Job Setup... Monitoring & Results... Conclusion... Tools & Platforms Used: Google Cloud Platform (GCP)... Vertex AI... Google Cloud Storage (GCS)... Gemini Models: Specifically gemini-2.0-flash-001... Python... Pandas Library... JSON Library... Web Browser... Data: Source: A combination of AI-generated prompts... Initial Format: CSV. Processing: Converted to JSONL... Final Format (JSONL)... Size: A small demo dataset... Target Audience: The video appears targeted towards developers, data scientists, or ML practitioners... Overall Impression: This video is a clear, well-structured, and practical walkthrough... Alright. We got the response from the Gemini 2.5 Pro. And you can see the analysis of this video. And let me check if it's doing correct and if it's really understanding my video contents. Okay, so the main topic... Yeah, it's demonstrating how to perform the supervised fine-tuning. And... I explained the different models of the Gemini for the API fine-tuning. And I explained the JSON formats and how to upload the data to the Cloud Storage, Google Cloud Storage. And checked... also showed how to do my personal project and preparing the data. Yeah, the target... Let's see the overall impression. So the video is clear, well structured, and practical walkthrough... Okay. Yeah, I think it's pretty much capturing all the contents I put in that video. So it's doing really great job. And I have to give a really good response for that. Um, okay.

**Cut Segment 5**
* **Timestamp:** [9:11 - 9:50]
* **Summary:** Transitions to the second part of the video, explaining the plan to use the Gemini 2.5 Pro Experimental model via its API within a Colab notebook, specifically demonstrating a practical project. References the API documentation shown earlier.
* **Transcript:** So I think next, um, what I'm really interested is to use, like, how do we do a personal, like project in Colab and to implement the API, how do we like really use the API requests in the Python notebook. So you can check this documentation for the API.

**Cut Segment 6**
* **Timestamp:** [11:31 - 12:39]
* **Summary:** Introduces the Colab notebook designed to rewrite prompts using the Gemini API (specifically `gemini-2.5-pro-exp-03-25` via `genai.Client`). Explains the script's function: read CSV, interact with the model using the client, send requests, handle rate limits (2 RPM / 50 per day free tier), and save results. Mentions prerequisites like an API key from Google AI Studio.
* **Transcript:** So this notebook will help you to walk through that. So what are we going to do is to basically read the CSV file and use the, uh, genai.client to interact with the Gemini 2.5 Pro model. And we're going to send the request using this function, the models.generate_contents with some retry logic and print the... Okay. So you have to have the Gemini API set up from the Google AI Studio. Yeah, you can create an API key.

**Cut Segment 7**
* **Timestamp:** [13:44 - 15:18]
* **Summary:** Explains the "Configuration & Client Initialization" and "Handling API Requests with Retries" sections of the Colab code. Shows setting the `MODEL_NAME` to `gemini-2.5-pro-exp-03-25`, initializing the `genai.Client`, and setting up rate limit constants (2 RPM, resulting in a 31-second delay). Shows the `send_request_with_retry` function using `client.models.generate_content`.
* **Transcript:** Okay, so this is the place we set up the client with... Okay, so this is the place we name the model and we set up the Gen AI client. And with some rate limit settings for now. And... So then I have, you know, request functions being written here to make sure that it won't cause any, um, rate limits issues. And this part is the real part for sending the request with the contents which will be the prompt in a list with the model name. And yes.

**Cut Segment 8**
* **Timestamp:** [15:32 - 17:13]
* **Summary:** Explains the "Preparing the Data and Constructing the Prompt" section of the Colab code. Shows the function to read the CSV data and the `rewrite_prompt` function, highlighting the specific prompt instructions given to the Gemini model to rewrite the original prompt while keeping all information and ensuring only the rewritten text is output.
* **Transcript:** So next part, we have the CSV file function, um, rewriting the prompt function. So this part is our, um, you know, prompt instructions for Gemini to, um, you know, execute the tasks for us. So like in our case, like we, we want Gemini to rewrite the prompt. And but we're going to, like, we're going to tell the Gemini that rewrite the following prompt while keeping all the original information except the same. And we insert the prompt from the Pandas data frame. And we also have some guidelines here. So we keep all the information and the meaning the same thing. Do not add or remove any details. Do not change anything. Since like that. And we must tell that provide only the rewritten prompt so that it won't, you know, response additional information. So eventually we want to add this rewritten prompt into our data frame. Okay. Next, we have to define our main function and execute all the codes.

**Cut Segment 9**
* **Timestamp:** [18:00 - 19:52]
* **Summary:** Demonstrates running the main execution cell in Colab. Shows the script starting, loading the 83 rows from `data.csv`, and beginning the prompt rewriting process. It displays Row 1's original prompt, the attempt to send the request to `gemini-2.5-pro-exp-03-25`, the successful response (rewritten prompt), and the enforced 31-second wait before processing the next row, confirming the rate limiting works as intended. Repeats for Row 2.
* **Transcript:** Alright. Make sure that you have the CSV file uploaded to the notebook, the Colab. And finally, we run the prompt rewriting process. And we read the file and do that. So here you can see from the results... and we load 83 rows from the data CSV file. And it starts processing that. And we can check that the original prompt... So we can, it's really nice to have the, you know, process being printed out. So we have, we can check that the model send the request to the Gemini model 2.5 Pro. And then we got received the prompt which is the rewritten, um, version of the original prompt. So, you know, it's waiting properly like the waiting for 31 seconds before the next request. And then it goes to the next one. And has original prompt and the received the rewritten prompt, assume the role... which is pretty cool.
```

In [ ]:
# @title 7. Clean Up (Optional)
def delete_file(client, file_obj):
    if file_obj:
        try:
            client.files.delete(name=file_obj.name)
            print(f"Deleted file: {file_obj.name}")
        except Exception as e:
            print(f"Error deleting file {file_obj.name}: {e}")

# Clean up both files
delete_file(client, video_file)
delete_file(client, ref_file)

Deleted file: files/99zfw6z8tkhr
Deleted file: files/1j6vm4i8dgpn
